# Ausreißer

Zunächst betrachten wir die *numerischen* Daten des Datensatzes. Diese kann man mit Hilfe von vorgegebenen Funktionen identifizieren. Darüberhinaus kann ein Blick in die Beschreibungsdatei des Datensatzes "data_description.txt" geworfen werden.

In [ ]:
#import some necessary librairies

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
print(check_output(["ls", "../data/house-prices-advanced-regression-techniques"]).decode("utf8")) #check the files available in the directory

import pickle

# Daten initial laden


In [ ]:
train = pd.read_csv('../data/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../data/house-prices-advanced-regression-techniques/test.csv')

print("Dimension des Trainingssets: " + str (train.shape))
print("Dimension des Testsets: " + str (test.shape))

# ID Spalten

In [ ]:
#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)


In [ ]:
# numerische Daten

numerical_features = train.select_dtypes(include=[np.number])
print ("Es gibt " + str(numerical_features.columns.size) + " numerische Spalten.\n")
for name in numerical_features.columns.sort_values():
    print (name)


In [ ]:
numerical_features.head(10)

In [ ]:
numerical_features.describe().T

# Ausreißer

Ausreißer sind Werte (einer Spalte), die stark von der "generellen Tendenz" der Mehrzahl der Daten abweichen. Dies kann sowohl für diskrete als auch für kontinuierliche numerische Daten gelten. Ausreißer können aufgrund verschiedener Ursachen entstehen:

* **Natürlich**: Hier handelt es sich um Ausreißer, die "in der Natur der Daten liegen" und gelten nicht als Fehler.
* **Eingabefehler**: Wenn Menschen (aber auch Maschinen, z. B. beim OCR) Daten erfassen, können Eingabefehler (Tippfehler, Erfassungsfehler, Ungenauigkeiten) vorliegen.
* **Messfehler**: Messungen sind immer nur so genau, wie ihr Sensor. Auch Sensoren können zufällig Ausreißer erzeugen.
* **Beschädigte Daten**: Bei der Verarbeitung (oder z. B. Konvertierung) von Daten können Fehler entstanden sein.

## Leseempfehlungen
[Sadrach Pierre, 2022, "How to Find Outliers (With Examples)"](https://builtin.com/data-science/how-find-outliers-examples)

[Schendera, C. 2007. Datenqualität mit SPSS. München, S. 163-174](https://www.degruyter.com/document/doi/10.1524/9783486710694/html?lang%253Dde)

# Umgang mit Ausreißern

Werden Daten - und deren Ausreißer - im maschinellen Lernen verarbeitet, hat dies Einfluss auf das Training, mithin also auf die Güte des Modells. Im ungünstigen Fall, wird die **Güte des Modells negativ beeinflusst**.

**Jedoch sind nicht alle Ausreißer negativ**, sondern bilden den Untersuchungsgegenstand ab. Werden z. B. Röntgenbilder durch ein Modell des maschinellen Lernens für das Trainig einer computergestützten Diagnostik eingesetzt, dann wird man z. B. nach prävalenten und nicht prävalenten Daten unterscheiden (Krankheit liegt vor | Krankeit liegt nicht vor). Ist die Krankheit selten auf den Bildern zu sehen, könnten diese als Ausreißer betrachtet werden, was nicht im Sinne der Modellierung ist.

Sie müssen sich also Fragen, ob die Ausreißer durch Fehler verursacht sind, oder in der "Natur der Sache" liegen.

=> **Ausreißer, die durch Fehler verursacht sind, _müssen_ entfernt werden.**
=> **Ausreißer, die den zu modellierenden Gegenstand abbilden, dürfen _nicht_ entfernt werden.**

## Überblick verschaffen
### Boxplots
Boxplots verwenden in der Standardeinstellung wie wir sie in der Abbildung sehen die Interquartile Range (IQR) - Methode für die Ausreißererkennung. Die IQR-Methode funktioniert wie folgt:
1. Das 25%-Quantil (Q1) berechnen
2. Das 75%-Quantil (Q3) berechnen
3. Q1 von Q3 subtrahieren (ergibt die Höhe der Box im Boxplot) = iqr
4. Die untere Grenze berechnen durch Q1 - (Faktor * iqr)
5. Die obere Grenze berechnen durch Q3 + (Faktor * iqr)

Wobei der Faktor variabel ist. Die Standardeinstellung beträgt 1,5.

In [ ]:
start = 0
anf = start
end = 0
while end < numerical_features.columns.size:
    end = anf + 5
    if end > numerical_features.columns.size:
        end = numerical_features.columns.size
    subset = numerical_features.iloc[:, anf:end]
    #print (min, max, subset.shape, subset.columns)
    subset.plot(kind="box",subplots=True,figsize=(15,5),title="Boxplots der numerischen Merkmale, Spalte " + str (anf) + " .. " + str (end -1));
    anf = end

#numerical_features.plot(kind="box",subplots=True,figsize=(150,25),title="Boxplots der numerischen Merkmale");


# Korrelationen

In [ ]:
# Eine Korrelationsmatrix (Correlation map) zeigt an, wie die einzelnen Features mit dem SalePrice korrelieren
corrmat = train.select_dtypes(include=[np.number]).corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

In [ ]:
#A better example (formatting used in below chart) - https://seaborn.pydata.org/examples/many_pairwise_correlations.html 

sns.set_theme(style="white")

# Korrelationsmatrix berechnen
corr = train.select_dtypes(include=[np.number]).corr()

# Wir erzeugen eine Maske für das obere Dreieck (sonst sieht es aus wie das Quadrat, das wir oben hatten, und ist überflüssig)
mask = np.triu(np.ones_like(corr, dtype=bool))

# Setup matplotlib Grafik
f, ax = plt.subplots(figsize=(25, 15))

# Wir erzeugen eine benutzerdefinierte divergierende Farbkarte
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Korrelationsmatrix zeichnen
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.8, cbar_kws={"shrink": .7}, annot=True, annot_kws={"fontsize":6})
#obviously many of thes variables are HIGHLY correlated. Something we may want to explore is why Average percentage viewed is negatively related to RPM

# Vgl. https://www.kaggle.com/code/kenjee/basic-eda-example-section-6

# Scatter Plot ausgesuchter Spalten

# Korrelation zw. der Größe und dem Verkaufspreis

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

# Löschen der Ausreißer

In [ ]:
#Deleting outliers
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)


In [ ]:
#Check the graphic again
fig, ax = plt.subplots()
ax.scatter(train['GrLivArea'], train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

# Korrelation zw. der "Overall Quality" und dem Verkaufspreis

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['OverallQual'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('OverallQual', fontsize=13)
plt.show()

Wir entscheiden uns hier, **keine** Ausreißer zu entfernen, da hier z. B. auch noch die Größe, Anzahl Zimmer etc. einfließt.

Wahrscheinlich gibt es noch weitere Ausreißer in den Trainingsdaten. Das Entfernen aller Ausreißer könnte sich jedoch negativ auf unsere Modelle auswirken, wenn es auch Ausreißer in den Testdaten gäbe. Deshalb werden wir nicht alle Ausreißer entfernen, sondern nur einige unserer Modelle für sie robust machen. Sie können sich dazu auf den Modellierungsteil dieses Notizbuchs beziehen.

Die Entfernung von Ausreißern ist nicht immer sicher. Wir werden lediglich zwei Spalten mit Ausreißern entfernen, da sie sehr groß und wirklich schlecht sind (extrem große Flächen zu sehr niedrigen Preisen).
 
[Documentation] Weiterführende Informationen zeigen verschiedene Ausreißer der Daten. [^1].

[^1]: http://ww2.amstat.org/publications/jse/v19n3/Decock/DataDocumentation.txt

In [ ]:
# Sichern der Zwischenergebnisse
with open('../data/house-prices-advanced-regression-techniques/train.pkl', 'wb') as handle:
    pickle.dump(train, handle)
with open('../data/house-prices-advanced-regression-techniques/test.pkl', 'wb') as handle:
    pickle.dump(test, handle)